# Exploratory Data Analysis for Steamit

### Importing Necessary Libraries

In [22]:
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer




# Step 6: Update the recommendation function to use the combined feature matrix
def recommend_games_combined(user_input, combined_feature_matrix, games_df):
    user_embedding = get_embedding(user_input)
    similarities = cosine_similarity(user_embedding, combined_feature_matrix)
    top_n = 5
    recommendations = similarities[0].argsort()[-top_n:][::-1]
    return recommendations

# Step 7: Use this function in your Streamlit app



# Connect to the database
conn = sqlite3.connect('steam_games.db')

# Create a cursor object
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Close the connection
conn.close()

# Print the list of tables
print("Tables in the database:", tables)


Tables in the database: [('game_details',), ('game_reviews',), ('sqlite_sequence',)]


In [23]:
# Correct table name based on your database structure
game_table = 'game_details'  # Replace with the actual table name if different
review_table = 'game_reviews'  # Replace with the actual table name if different

# Reconnect to the database
conn = sqlite3.connect('steam_games.db')

# Load the data into pandas DataFrames
games_df = pd.read_sql_query(f"SELECT * FROM {game_table}", conn)
reviews_df = pd.read_sql_query(f"SELECT * FROM {review_table}", conn)

# Close the connection
conn.close()

# Display the first few rows of the dataframes
games_df.head()



,appid,name,description,price,release_date,developer,publisher,tags
0,1396670,Fantasy Grounds - Pathfinder RPG - Wrath of th...,Pathfinder RPG - Wrath of the Righteous AP 1: ...,₹ 569,"29 Sep, 2020","SmiteWorks USA, LLC",,"Indie, RPG, Strategy"
1,1396680,Fantasy Grounds - Pathfinder RPG - Wrath of th...,Pathfinder RPG - Wrath of the Righteous AP 2: ...,₹ 569,"20 Apr, 2021","SmiteWorks USA, LLC",,"Indie, RPG, Strategy"
2,1396690,Fantasy Grounds - Pathfinder RPG - Wrath of th...,Pathfinder RPG - Wrath of the Righteous AP 3: ...,₹ 569,"11 May, 2021","SmiteWorks USA, LLC",,"Indie, RPG, Strategy"
3,1396700,Fantasy Grounds - Pathfinder RPG - Wrath of th...,Pathfinder RPG - Wrath of the Righteous AP 4: ...,₹ 569,"28 Sep, 2021","SmiteWorks USA, LLC",,"Indie, RPG, Strategy"
4,1396710,Fantasy Grounds - Pathfinder RPG - Wrath of th...,Pathfinder RPG - Wrath of the Righteous AP 5: ...,₹ 569,"21 Dec, 2021","SmiteWorks USA, LLC",,"Indie, RPG, Strategy"


In [24]:
# Remove DLCs, Playtests, and Demos from games_df
filtered_games_df = games_df[~games_df['name'].str.contains('soundtrack|OST|demo|DLC|playtest|resource pack', case=False, na=False)]
filtered_games_df.to_csv('filtered_games_df.csv', index=False)
print(filtered_games_df.head(), filtered_games_df.shape)

     appid                                               name  \
0  1396670  Fantasy Grounds - Pathfinder RPG - Wrath of th...   
1  1396680  Fantasy Grounds - Pathfinder RPG - Wrath of th...   
3  1396700  Fantasy Grounds - Pathfinder RPG - Wrath of th...   
4  1396710  Fantasy Grounds - Pathfinder RPG - Wrath of th...   
5  1396720  Fantasy Grounds - Pathfinder RPG - Wrath of th...   

                                         description  price  release_date  \
0  Pathfinder RPG - Wrath of the Righteous AP 1: ...  ₹ 569  29 Sep, 2020   
1  Pathfinder RPG - Wrath of the Righteous AP 2: ...  ₹ 569  20 Apr, 2021   
3  Pathfinder RPG - Wrath of the Righteous AP 4: ...  ₹ 569  28 Sep, 2021   
4  Pathfinder RPG - Wrath of the Righteous AP 5: ...  ₹ 569  21 Dec, 2021   
5  Pathfinder RPG - Wrath of the Righteous AP 6: ...  ₹ 569  11 Jan, 2022   

             developer publisher                  tags  
0  SmiteWorks USA, LLC            Indie, RPG, Strategy  
1  SmiteWorks USA, LLC          

In [25]:
# Filter reviews based on the filtered games_df appid
filtered_reviews_df = reviews_df[reviews_df['appid'].isin(filtered_games_df['appid'])]
print("Filtered reviews_df shape:", filtered_reviews_df.shape)

Filtered reviews_df shape: (51804, 8)


In [26]:
# Load pre-trained BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Function to get text embedding
def get_embedding(text):
    inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().detach().numpy()

# Generate embeddings for all game descriptions
embeddings = []
for description in filtered_games_df['description']:
    embeddings.append(get_embedding(description).flatten())

# Convert to numpy array and save
bert_item_feature_matrix = np.array(embeddings)
np.save('bert_item_feature_matrix.npy', bert_item_feature_matrix)
print("Item feature matrix has been successfully saved as 'bert_item_feature_matrix.npy")

C:\Users\Jadoo\Documents\CSProjects\steam_recomendation_system\Steamlit\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contain

Item feature matrix has been successfully saved as 'bert_item_feature_matrix.npy


In [27]:
# Step 1: Vectorize the reviews using CountVectorizer for LDA
import re

# Apply the clean_text function to the reviews
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip()  # Strip leading and trailing spaces
    return text

filtered_reviews_df = reviews_df[reviews_df['appid'].isin(filtered_games_df['appid'])]
filtered_reviews_df['cleaned_text'] = filtered_reviews_df['review_text'].apply(clean_text)

vectorizer = CountVectorizer(max_features=5000, stop_words='english')
reviews_vectorized = vectorizer.fit_transform(filtered_reviews_df['cleaned_text'])

# Step 2: Fit the LDA model (using 20 topics as an example)
lda_model = LatentDirichletAllocation(n_components=20, random_state=42)
lda_topic_matrix = lda_model.fit_transform(reviews_vectorized)

# Step 3: Create a new DataFrame to store LDA topics and app_id
lda_df = pd.DataFrame(lda_topic_matrix, columns=[f'topic_{i}' for i in range(lda_topic_matrix.shape[1])])
lda_df['appid'] = filtered_reviews_df['appid'].values

# Step 4: Aggregate the LDA topics per game (group by appid)
filtered_reviews_df['lda_topics'] = lda_topic_matrix.tolist()
lda_topic_matrix_per_game = filtered_reviews_df.groupby('appid')['lda_topics'].apply(lambda x: np.mean(x.tolist(), axis=0)).to_list()

# Convert lda_topic_matrix_per_game to a 2D numpy array
lda_topic_matrix_per_game = np.array(lda_topic_matrix_per_game)

# Step 5: Filter bert_item_feature_matrix to match the LDA topic matrix appids
matching_appids = filtered_games_df['appid'].isin(filtered_reviews_df['appid'].unique())
filtered_games_df = filtered_games_df[matching_appids]
bert_item_feature_matrix = bert_item_feature_matrix[matching_appids]

# Step 6: Combine the matrices
combined_feature_matrix = np.hstack((bert_item_feature_matrix, lda_topic_matrix_per_game))

# Step 7: Check the combined matrix dimensions
print("Combined feature matrix shape:", combined_feature_matrix.shape)

C:\Users\Jadoo\AppData\Local\Temp\ipykernel_16772\1065448926.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews_df['cleaned_text'] = filtered_reviews_df['review_text'].apply(clean_text)
C:\Users\Jadoo\AppData\Local\Temp\ipykernel_16772\1065448926.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews_df['lda_topics'] = lda_topic_matrix.tolist()


Combined feature matrix shape: (5889, 788)


# Test on BERT Embeddings + LDA

In [28]:
combined_feature_matrix.shape

(5889, 788)

In [29]:
# Step 7: Define a quick recommendation function
from sklearn.metrics.pairwise import cosine_similarity
# Function to get user embedding including LDA topics
def get_combined_user_embedding(user_input, bert_model, lda_model, vectorizer):
    # Get BERT embedding
    user_embedding = get_embedding(user_input)
    
    # Get LDA topic distribution for user input
    user_input_vectorized = vectorizer.transform([user_input])
    user_lda_topics = lda_model.transform(user_input_vectorized)
    
    # Combine BERT embedding and LDA topics
    combined_user_embedding = np.hstack((user_embedding.flatten(), user_lda_topics.flatten()))
    
    return combined_user_embedding.reshape(1, -1)

# Updated recommendation function
def recommend_games_combined(user_input, combined_feature_matrix, games_df):
    combined_user_embedding = get_combined_user_embedding(user_input, bert_model, lda_model, vectorizer)
    similarities = cosine_similarity(combined_user_embedding, combined_feature_matrix)
    top_n = 5
    recommendations = similarities[0].argsort()[-top_n:][::-1]
    return recommendations

# Example user input
user_input = "I want a challenging fantasy game with history"
recommendations = recommend_games_combined(user_input, combined_feature_matrix, filtered_games_df)

# Print the recommended games
for idx in recommendations:
    print(filtered_games_df.iloc[idx]['name'])

Adventure Minesweeper
Drabidiboo
DIERY HEAVEN
Forest Keeper
Archer Master
